In [95]:
import pandas as pd
import nltk
from nltk import word_tokenize
from wikitables import import_tables
import wikipedia as wp
import pprint
from nltk.corpus import wordnet as wn
from itertools import chain
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.metrics import precision_score , recall_score
from scipy import sparse

In [7]:
wiki_sheet = pd.read_csv('Wikipedia_Tables_one.csv',encoding='cp1252')

In [8]:
wk_tables = wiki_sheet

In [9]:
wk_tables.head(12)

,S.No,Name of Table,Column Names,Unnamed: 3,Member,Measure,Root,Other Measures,Missing range,URL,comments,R/F
0,1,List of largest California cities by land area,"Rank, City, Land Area, Cum Area, Total Area, P...",NaN,City,Land Area,cities,"Cum Area, Total Area, Population, Population D...",NO,https://en.wikipedia.org/wiki/List_of_largest_...,NaN,R
1,2,List of the 100 largest cities and towns in Ca...,"Rank, Municipality, Province ,Status , Area",NaN,Municipality,Area,cities,Province,NO,https://en.wikipedia.org/wiki/List_of_the_100_...,NaN,R
2,3,List of city squares by size,"City Square, City, Country, Area, Dimensions; ...",NaN,City Square,Area,squares,NaN,YES,https://en.wikipedia.org/wiki/List_of_city_squ...,NaN,R
3,4,List of countries and dependencies by area,"Rank, Sovereign State/ dependency, Total in km...",NaN,Sovereign State/ dependency,Total in km,countries,NaN,YES,https://en.wikipedia.org/wiki/List_of_countrie...,NaN,R
4,5,List of countries and territories by border/ar...,"Rank, Country or Territory, Total length of la...",NaN,Country or Teritory,Border/ area ratio,countries,NaN,NO,https://en.wikipedia.org/wiki/List_of_countrie...,NaN,R
5,6,List of countries by irrigated land area,"Rank, Country/ Region, Irrigated Land, Date of...",NaN,Country/ Region,Irrigated Land,countries,NaN,NO,https://en.wikipedia.org/wiki/List_of_countrie...,NaN,R
6,7,List of largest empires,"Empire, Maximum Land Area",NaN,Empire,Maximum Land Area,empires,NaN,NO,https://en.wikipedia.org/wiki/List_of_largest_...,NaN,R
7,8,List of largest empires in India,"Empire, Approximate Maximum Extent, Size as\np...",NaN,Empire,Approximate Maximum Extent,empires,NaN,NO,https://en.wikipedia.org/wiki/List_of_largest_...,NaN,R
8,9,List of European countries by area,"Rank, State, Total area (km2),Notes",NaN,State,Total area (km2),countries,NaN,NO,https://en.wikipedia.org/wiki/List_of_European...,NaN,R
9,10,List of cities in India by area,"\nRank, City, State / UT, Area (km2), Area gov...",NaN,City,Area,cities,NaN,NO,https://en.wikipedia.org/wiki/List_of_cities_i...,NaN,F


In [10]:
wk_tables.shape

(93, 12)

<h1> Extracting Tables </h1>

<h2> POS TAGGING </h2>

In [11]:
member_attribute_list = list(wk_tables['Member'])

In [12]:
member_attribute_list[2] = 'City square' 
member_attribute_list[3] = 'Sovereign state/dependency'
member_attribute_list[4] = 'Country or territory'
member_attribute_list[5] = 'Country/Region'

In [13]:
title_list = list(wk_tables['Name of Table'])

In [14]:
title_list[19]

'List of largest cruise ships'

<h1> List of Titles</h1>

In [15]:
title_list

['List of largest California cities by land area',
 'List of the 100 largest cities and towns in Canada by area',
 'List of city squares by size',
 'List of countries and dependencies by area',
 'List of countries and territories by border/area ratio',
 'List of countries by irrigated land area',
 'List of largest empires',
 'List of largest empires in India',
 'List of European countries by area',
 'List of cities in India by area',
 'List of United States cities by area',
 'List of Oceanian countries by area',
 'List of largest cities by area',
 'List of U.S. states and territories by area',
 'List of largest Internet companies',
 'List of most-retweeted tweets',
 'List of most popular websites',
 'List of most-followed Twitter accounts',
 'List of most expensive domain names',
 'List of largest cruise ships',
 'List of oldest cats',
 'List of largest LGBT events',
 'Largest fungal fruit bodies',
 'List of largest cities in the Arab world',
 'List of largest seeds',
 'List of oldest 

In [16]:
title_list_ref = []
for t in title_list:
    try:
        title_list_ref.append(t.replace('List', ''))
    except:
        title_list_ref.append(t)

In [17]:
pos_title_tags = []
for t in title_list_ref:
    print(t)
    tags = word_tokenize(t)
    pos_title_tags.append(nltk.pos_tag(tags))

 of largest California cities by land area
 of the 100 largest cities and towns in Canada by area
 of city squares by size
 of countries and dependencies by area
 of countries and territories by border/area ratio
 of countries by irrigated land area
 of largest empires
 of largest empires in India
 of European countries by area
 of cities in India by area
 of United States cities by area
 of Oceanian countries by area
 of largest cities by area
 of U.S. states and territories by area
 of largest Internet companies
 of most-retweeted tweets
 of most popular websites
 of most-followed Twitter accounts
 of most expensive domain names
 of largest cruise ships
 of oldest cats
 of largest LGBT events
Largest fungal fruit bodies
 of largest cities in the Arab world
 of largest seeds
 of oldest trees
 of the oldest newspapers
 of elevation extremes by region
 of the largest villages in England
 of largest rough diamonds
 of dictionaries by number of words
 of domesticated animals
 of most-prod

In [18]:
pos_title_tags[1]

[('of', 'IN'),
 ('the', 'DT'),
 ('100', 'CD'),
 ('largest', 'JJS'),
 ('cities', 'NNS'),
 ('and', 'CC'),
 ('towns', 'NNS'),
 ('in', 'IN'),
 ('Canada', 'NNP'),
 ('by', 'IN'),
 ('area', 'NN')]

In [19]:
len(pos_title_tags)

93

<h1> Attribute Superlative Phrase </h1>

<h2> Extracting Noun Phrase Along with it Simultaneously</h2>

In [20]:
noun_phrases = {}
sup_phrases = []
i = 0
for v in range (len(pos_title_tags)):
    print(i)
    count_sup = 0
    n_p =[]
    for k in range(len(pos_title_tags[v])):
        if pos_title_tags[v][k][1] == 'JJS':
            print(pos_title_tags[v][k])
            count_sup = 1
            sup_phrases.append(pos_title_tags[v][k][0])
        elif pos_title_tags[v][k][1] == 'NNS':
            n_p.append(pos_title_tags[v][k][0])
        elif pos_title_tags[v][k][1] == 'NNP':
            n_p.append(pos_title_tags[v][k][0])
        elif pos_title_tags[v][k][1] == 'NN':
            n_p.append(pos_title_tags[v][k][0])
    noun_phrases[i] = n_p
    if count_sup ==0:
            sup_phrases.append('')
    i = i+1

0
('largest', 'JJS')
1
('largest', 'JJS')
2
3
4
5
6
('largest', 'JJS')
7
('largest', 'JJS')
8
9
10
11
12
('largest', 'JJS')
13
14
('largest', 'JJS')
15
16
('most', 'JJS')
17
18
('most', 'JJS')
19
('largest', 'JJS')
20
('oldest', 'JJS')
21
('largest', 'JJS')
22
23
('largest', 'JJS')
24
('largest', 'JJS')
25
('oldest', 'JJS')
26
('oldest', 'JJS')
27
28
('largest', 'JJS')
29
('largest', 'JJS')
30
31
32
33
('most', 'JJS')
34
('heaviest', 'JJS')
35
36
('longest', 'JJS')
37
('fastest', 'JJS')
38
('fastest', 'JJS')
39
40
('largest', 'JJS')
41
('largest', 'JJS')
42
('largest', 'JJS')
43
('longest', 'JJS')
44
45
('smallest', 'JJS')
46
47
('largest', 'JJS')
48
('longest', 'JJS')
49
('longest', 'JJS')
50
('wealthiest', 'JJS')
51
52
('largest', 'JJS')
53
('most', 'JJS')
54
55
56
57
58
('longest', 'JJS')
59
('greatest', 'JJS')
60
61
('wealthiest', 'JJS')
62
63
64
65
66
67
68
69
70
71
72
73
74
75
('largest', 'JJS')
76
('highest', 'JJS')
77
('highest', 'JJS')
78
('most', 'JJS')
79
('largest', 'JJS')


In [21]:
sup_phrases

['largest',
 'largest',
 '',
 '',
 '',
 '',
 'largest',
 'largest',
 '',
 '',
 '',
 '',
 'largest',
 '',
 'largest',
 '',
 'most',
 '',
 'most',
 'largest',
 'oldest',
 'largest',
 '',
 'largest',
 'largest',
 'oldest',
 'oldest',
 '',
 'largest',
 'largest',
 '',
 '',
 '',
 'most',
 'heaviest',
 '',
 'longest',
 'fastest',
 'fastest',
 '',
 'largest',
 'largest',
 'largest',
 'longest',
 '',
 'smallest',
 '',
 'largest',
 'longest',
 'longest',
 'wealthiest',
 '',
 'largest',
 'most',
 '',
 '',
 '',
 '',
 'longest',
 'greatest',
 '',
 'wealthiest',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'largest',
 'highest',
 'highest',
 'most',
 'largest',
 '',
 '',
 'tallest',
 'heaviest',
 'youngest',
 'youngest',
 'youngest',
 'youngest',
 'oldest',
 'largest',
 'tallest',
 'youngest',
 'youngest']

In [22]:
wk_tables['sup_label'] = sup_phrases

In [23]:
wk_tables.to_csv('check_labels.csv')

In [117]:
noun_phrases[1]

['cities', 'towns', 'Canada', 'area']

In [26]:
len(sup_phrases)

93

In [27]:
len(noun_phrases)

93

<h1> Attribute Root Words List Obtained from Syntax Net Parser</h1>

In [28]:
#root_word_list = ['cities','cities','squares','countries','countries','countries','empires','empires',
#                 'countries','cities'] #obtained from syntaxnet parser

In [29]:
root_word_list = list(wk_tables['Root'])

In [30]:
len(root_word_list)

93

<h1>Attribute Hypernyms</h1>

This is obtained by performing Named Entity Recognition on the values of the member attribute as well as the noun phrases extracted from the above POS Tagging step

In [31]:
member_attribute_list[54] ='Temperature °C'
member_attribute_list[55] ='Temperature'
member_attribute_list[56] = 'Region\n'
member_attribute_list[63] ='Prefectures'
member_attribute_list[76] ='Goals  scored'
member_attribute_list[77] ='Goals  scored'
member_attribute_list[86] ='Name'
member_attribute_list[83] ='Name'
member_attribute_list[87] ='Name'
#member_attribute_list[93] ='Time as youngest senator'
member_attribute_list[90] = 'Rank'
member_attribute_list[91] = 'Age at time of murder(s)'

<h2> Getting Values from Member Attribute and also obtaining column names </h2>

In [32]:
column_names = {}
member_values_str = []
member_vals_array = {}
i = 0
for values in title_list:
    vals = []
    tables = import_tables(str(values))
    #print(values)
    column_names[i] = tables[0].head
    get_str = member_attribute_list[i]
    print(i)
    #print(get_str)
    for row in tables[0].rows:
        check = row[get_str]
        break 
    for row in tables[0].rows:
        try:
            vals.append(row[get_str])
        except:
            print(i)
            print(' ')
    final_vals = ' '.join(str(e) for e in vals)
    member_vals_array[i] = vals
    member_values_str.append(final_vals)
    i = i+1

List of largest California cities by land area[0][12]: missing field for column [Population Density]
List of largest California cities by land area[0][13]: missing field for column [Population Density]
List of largest California cities by land area[0][14]: missing field for column [Population Density]
List of largest California cities by land area[0][15]: missing field for column [Population Density]
List of largest California cities by land area[0][16]: missing field for column [Population Density]
List of largest California cities by land area[0][17]: missing field for column [Population Density]
List of largest California cities by land area[0][18]: missing field for column [Population Density]
List of largest California cities by land area[0][19]: missing field for column [Population Density]
List of largest California cities by land area[0][20]: missing field for column [Population Density]
List of largest California cities by land area[0][21]: missing field for column [Population

0
1


List of city squares by size[0][0]: missing field for column [Country]
List of city squares by size[0][0]: missing field for column [Area]
List of city squares by size[0][0]: missing field for column [Dimensions; Notes]
List of city squares by size[0][0]: missing field for column [Yearcompleted]
List of city squares by size[0][0]: missing field for column [Photo]
List of city squares by size[0][31]: missing field for column [Photo]
List of city squares by size[0][35]: missing field for column [Photo]
List of city squares by size[0][37]: missing field for column [Photo]
List of city squares by size[0][39]: missing field for column [Photo]
List of city squares by size[0][40]: missing field for column [Photo]
List of city squares by size[0][45]: missing field for column [Photo]
List of city squares by size[0][53]: missing field for column [Photo]
List of city squares by size[0][57]: missing field for column [Photo]
List of city squares by size[0][58]: missing field for column [Photo]
List

2
3
4
5


List of largest empires[0][0]: dropping field from unknown column: % of world
List of largest empires[0][0]: dropping field from unknown column: Year
List of largest empires[0][1]: dropping field from unknown column: 35.5 km2 mi2 2
List of largest empires[0][1]: dropping field from unknown column: 35.5 134.74 2
List of largest empires[0][1]: dropping field from unknown column: 1920
List of largest empires[0][2]: dropping field from unknown column: 24.0 km2 mi2 2
List of largest empires[0][2]: dropping field from unknown column: 24.0 134.74 2
List of largest empires[0][2]: dropping field from unknown column: 1270 or 1309
List of largest empires[0][3]: dropping field from unknown column: 22.8 km2 mi2 2
List of largest empires[0][3]: dropping field from unknown column: 22.8 134.74 2
List of largest empires[0][3]: dropping field from unknown column: 1895
List of largest empires[0][4]: dropping field from unknown column: 13.7 km2 mi2 2
List of largest empires[0][4]: dropping field from unkn

...

List of largest lakes of Europe[0][0]: missing field for column [Notes]
List of largest lakes of Europe[0][1]: missing field for column [Notes]
List of largest lakes of Europe[0][2]: missing field for column [Notes]
List of largest lakes of Europe[0][3]: missing field for column [Notes]
List of largest lakes of Europe[0][4]: missing field for column [Notes]
List of largest lakes of Europe[0][5]: missing field for column [Notes]
List of largest lakes of Europe[0][6]: missing field for column [Notes]
List of largest lakes of Europe[0][7]: missing field for column [Notes]
List of largest lakes of Europe[0][8]: missing field for column [Notes]
List of largest lakes of Europe[0][9]: missing field for column [Notes]
List of largest lakes of Europe[0][10]: missing field for column [Notes]
List of largest lakes of Europe[0][11]: missing field for column [Notes]
List of largest lakes of Europe[0][12]: missing field for column [Notes]
List of largest lakes of Europe[0][13]: missing field for col

List of largest lakes of Europe[0][105]: missing field for column [Notes]
List of largest lakes of Europe[0][106]: missing field for column [Notes]
List of largest lakes of Europe[0][107]: missing field for column [Notes]
List of largest lakes of Europe[0][108]: missing field for column [Notes]
List of largest lakes of Europe[0][109]: missing field for column [Notes]
List of largest lakes of Europe[0][110]: missing field for column [Notes]
List of largest lakes of Europe[0][111]: missing field for column [Notes]
List of largest lakes of Europe[0][112]: missing field for column [Notes]
List of largest lakes of Europe[0][113]: missing field for column [Notes]
List of largest lakes of Europe[0][114]: missing field for column [Notes]
List of largest lakes of Europe[0][115]: missing field for column [Notes]
List of largest lakes of Europe[0][116]: missing field for column [Notes]
List of largest lakes of Europe[0][117]: missing field for column [Notes]
List of largest lakes of Europe[0][118

89


List of tallest statues[1][7]: missing field for column [Statue]
List of tallest statues[1][7]: missing field for column [Photo]
List of tallest statues[1][7]: missing field for column [Depicts]
List of tallest statues[1][7]: missing field for column [Locations]
List of tallest statues[1][7]: missing field for column [Country]
List of tallest statues[1][7]: missing field for column [Heightm (ft)]
List of tallest statues[1][7]: missing field for column [Completed]
List of tallest statues[1][7]: missing field for column [Destroyed]
List of tallest statues[1][7]: missing field for column [Cause of destruction]
List of tallest statues[1][7]: missing field for column [Notes]
List of tallest statues[1][7]: missing field for column [Ref]


90
91
92


In [33]:
tables = import_tables('List of most-viewed YouTube videos')

List of most-viewed YouTube videos[0][30]: missing field for column [Video name]
List of most-viewed YouTube videos[0][30]: missing field for column [Uploader / artist]
List of most-viewed YouTube videos[0][30]: missing field for column [Views (billions)]
List of most-viewed YouTube videos[0][30]: missing field for column [Upload date]
List of most-viewed YouTube videos[0][30]: missing field for column [Note]
List of most-viewed YouTube videos[1][5]: missing field for column [Upload date]
List of most-viewed YouTube videos[1][5]: missing field for column [Notes]
List of most-viewed YouTube videos[1][11]: missing field for column [Upload date]
List of most-viewed YouTube videos[1][11]: missing field for column [Notes]
List of most-viewed YouTube videos[1][17]: missing field for column [Upload date]
List of most-viewed YouTube videos[1][17]: missing field for column [Notes]
List of most-viewed YouTube videos[1][23]: missing field for column [Upload date]
List of most-viewed YouTube video

In [36]:
tables[0].head

['#',
 'Video name',
 'Uploader / artist',
 'Views (billions)',
 'Upload date',
 'Note']

In [102]:
tables = import_tables('List of pathogens by size')
print(tables[0].head)
for row in tables[0].rows:
    print(row['Classification'],row['Binomial name'])

['Classification', 'Binomial name', 'Common name', 'Dimension', 'Size', 'Reference']
Flatworm animal Eucestoda
Nematode animal Loa loa
Arthropod animal Cymothoa exigua
Nematode animal Enterobius
Arthropod animal Pediculus humanus capitis
Flatworm animal Eucestoda
Arthropod animal Sarcoptes scabiei
Metamonad Giardia lamblia
Fungus Candida albicans
Bacterium Borrelia burgdorferi
Alveolate protist Plasmodium
Fungus [many]
Bacterium Yersinia pestis
Bacterium Mycobacterium tuberculosis
Alveolate protist Plasmodium
Bacterium Mycoplasma genitalium
Virus HIV
Virus Orthomyxoviridae
Virus Rhinovirus
Virus Porcine circovirus


In [37]:
member_attribute_list[23]

'City'

In [38]:
for row in tables[0].rows:
    print(row['Video name'])

" Despacito "
" Baby Shark Dance "
" Shape of You "
" See You Again "
''Masha and the Bear'' – "Recipe for Disaster"
" Uptown Funk "
" Gangnam Style "
" Sorry "
" Sugar "
" Johny Johny Yes Papa "
"Learning Colors - Colorful Eggs on a Farm"
" Roar "
" Thinking Out Loud "
" Counting Stars "
" Shake It Off "
" Bailando "
" Lean On "
" Dark Horse "
" Faded "
" Let Her Go "
" Mi Gente "
" Girls Like You "
" Hello "
" Blank Space "
" Chantaje "
"Bath Song"
" Closer (Lyric video)"
" Waka Waka (This Time for Africa) "
"Phonics Song with Two Words"
" We Don't Talk Anymore "


KeyError: 'Video name'

In [39]:
title_list[26]

'List of the oldest newspapers'

In [40]:
for row in tables[0].rows:
    print(row['Rank'])

KeyError: 'Rank'

In [41]:
member_vals_array[0]

[Los Angeles,
 San Diego,
 California City,
 San Jose,
 Bakersfield,
 Fresno,
 Palmdale,
 Sacramento,
 Lancaster,
 Palm Springs,
 Riverside,
 Fremont,
 Apple Valley,
 Victorville,
 Hesperia,
 Irvine,
 Stockton,
 Redding,
 San Bernardino,
 Twentynine Palms,
 Adelanto,
 Oakland,
 Thousand Oaks,
 Santa Clarita,
 Moreno Valley]

In [42]:
#for row in tables[0].rows:
 #   print(row['Song'])

In [43]:
len(member_values_str)

93

<h1> Obtaining hypernyms </h1>

In [44]:
hypernyms_list = {}

In [45]:
h_counter = 0
for i in range(len(title_list_ref)):
    h_list = []
    for v in member_vals_array[i]:  #for obtaining hypernyms from member entities
        for i,j in enumerate(wn.synsets(str(v))):
            print ("Meaning",i, "NLTK ID:", j.name())
            print ("Hypernyms:", ", ".join(list(chain(*[l.lemma_names() for l in j.hypernyms()]))))
            ll = (list(chain(*[l.lemma_names() for l in j.hypernyms()])))
            h_list.append(ll)
    for v in noun_phrases[i]:     #for obtaining hypernyms from noun_phrases 
        for i,j in enumerate(wn.synsets(v)):
            print ("Meaning",i, "NLTK ID:", j.name())
            print ("Hypernyms:", ", ".join(list(chain(*[l.lemma_names() for l in j.hypernyms()]))))
            ll = (list(chain(*[l.lemma_names() for l in j.hypernyms()])))
            h_list.append(ll)
    hypernyms_list[h_counter] = h_list
    h_counter = h_counter+1

Meaning 0 NLTK ID: bakersfield.n.01
Hypernyms: 
Meaning 0 NLTK ID: fresno.n.01
Hypernyms: 
Meaning 0 NLTK ID: sacramento.n.01
Hypernyms: 
Meaning 0 NLTK ID: lancaster.n.01
Hypernyms: 
Meaning 1 NLTK ID: lancaster.n.02
Hypernyms: dynasty, royalty, royal_family, royal_line, royal_house
Meaning 0 NLTK ID: riverbank.n.01
Hypernyms: bank
Meaning 1 NLTK ID: riverside.n.02
Hypernyms: 
Meaning 0 NLTK ID: fremont.n.01
Hypernyms: 
Meaning 0 NLTK ID: stockton.n.01
Hypernyms: 
Meaning 0 NLTK ID: redding.n.01
Hypernyms: 
Meaning 0 NLTK ID: oakland.n.01
Hypernyms: 
Meaning 0 NLTK ID: california.n.01
Hypernyms: 
Meaning 0 NLTK ID: city.n.01
Hypernyms: municipality
Meaning 1 NLTK ID: city.n.02
Hypernyms: administrative_district, administrative_division, territorial_division
Meaning 2 NLTK ID: city.n.03
Hypernyms: municipality
Meaning 0 NLTK ID: land.n.01
Hypernyms: real_property, real_estate, realty, immovable
Meaning 1 NLTK ID: land.n.02
Hypernyms: object, physical_object
Meaning 2 NLTK ID: domain.n.

Meaning 0 NLTK ID: germany.n.01
Hypernyms: 
Meaning 0 NLTK ID: finland.n.01
Hypernyms: 
Meaning 0 NLTK ID: vietnam.n.01
Hypernyms: 
Meaning 1 NLTK ID: vietnam_war.n.01
Hypernyms: 
Meaning 0 NLTK ID: malaysia.n.01
Hypernyms: 
Meaning 0 NLTK ID: norway.n.01
Hypernyms: 
Meaning 0 NLTK ID: poland.n.01
Hypernyms: 
Meaning 0 NLTK ID: oman.n.01
Hypernyms: 
Meaning 0 NLTK ID: italy.n.01
Hypernyms: 
Meaning 0 NLTK ID: philippines.n.01
Hypernyms: 
Meaning 1 NLTK ID: philippines.n.02
Hypernyms: 
Meaning 2 NLTK ID: philippine.n.01
Hypernyms: Western_Malayo-Polynesian
Meaning 0 NLTK ID: ecuador.n.01
Hypernyms: 
Meaning 0 NLTK ID: gabon.n.01
Hypernyms: 
Meaning 0 NLTK ID: guinea.n.01
Hypernyms: coin
Meaning 1 NLTK ID: wop.n.01
Hypernyms: Italian
Meaning 2 NLTK ID: guinea.n.03
Hypernyms: 
Meaning 3 NLTK ID: guinea_fowl.n.01
Hypernyms: domestic_fowl, fowl, poultry
Meaning 0 NLTK ID: uganda.n.01
Hypernyms: 
Meaning 0 NLTK ID: ghana.n.01
Hypernyms: 
Meaning 0 NLTK ID: romania.n.01
Hypernyms: 
Meaning 0 

Meaning 0 NLTK ID: paraguay.n.01
Hypernyms: 
Meaning 0 NLTK ID: gabon.n.01
Hypernyms: 
Meaning 0 NLTK ID: malaysia.n.01
Hypernyms: 
Meaning 0 NLTK ID: thailand.n.01
Hypernyms: 
Meaning 0 NLTK ID: nicaragua.n.01
Hypernyms: 
Meaning 0 NLTK ID: greece.n.01
Hypernyms: 
Meaning 1 NLTK ID: greece.n.02
Hypernyms: Balkan_country, Balkan_nation, Balkan_state
Meaning 0 NLTK ID: poland.n.01
Hypernyms: 
Meaning 0 NLTK ID: ghana.n.01
Hypernyms: 
Meaning 0 NLTK ID: tunisia.n.01
Hypernyms: 
Meaning 0 NLTK ID: myanmar.n.01
Hypernyms: 
Meaning 0 NLTK ID: uruguay.n.01
Hypernyms: 
Meaning 0 NLTK ID: pakistan.n.01
Hypernyms: 
Meaning 0 NLTK ID: afghanistan.n.01
Hypernyms: 
Meaning 0 NLTK ID: iraq.n.01
Hypernyms: 
Meaning 0 NLTK ID: chile.n.01
Hypernyms: 
Meaning 1 NLTK ID: chili.n.02
Hypernyms: hot_pepper
Meaning 0 NLTK ID: finland.n.01
Hypernyms: 
Meaning 0 NLTK ID: norway.n.01
Hypernyms: 
Meaning 0 NLTK ID: zimbabwe.n.01
Hypernyms: 
Meaning 0 NLTK ID: ecuador.n.01
Hypernyms: 
Meaning 0 NLTK ID: ukraine.

Meaning 3 NLTK ID: sphere.n.01
Hypernyms: environment
Meaning 4 NLTK ID: area.n.05
Hypernyms: structure, construction
Meaning 5 NLTK ID: area.n.06
Hypernyms: extent
Meaning 0 NLTK ID: china.n.01
Hypernyms: 
Meaning 1 NLTK ID: china.n.02
Hypernyms: porcelain
Meaning 2 NLTK ID: taiwan.n.01
Hypernyms: 
Meaning 3 NLTK ID: chinaware.n.01
Hypernyms: crockery, dishware
Meaning 0 NLTK ID: india.n.01
Hypernyms: 
Meaning 0 NLTK ID: pakistan.n.01
Hypernyms: 
Meaning 0 NLTK ID: iran.n.01
Hypernyms: 
Meaning 0 NLTK ID: indonesia.n.01
Hypernyms: 
Meaning 0 NLTK ID: mexico.n.01
Hypernyms: 
Meaning 0 NLTK ID: thailand.n.01
Hypernyms: 
Meaning 0 NLTK ID: brazil.n.01
Hypernyms: 
Meaning 1 NLTK ID: brazil_nut.n.02
Hypernyms: edible_nut
Meaning 0 NLTK ID: bangladesh.n.01
Hypernyms: 
Meaning 0 NLTK ID: turkey.n.01
Hypernyms: domestic_fowl, fowl, poultry
Meaning 1 NLTK ID: turkey.n.02
Hypernyms: 
Meaning 2 NLTK ID: joker.n.02
Hypernyms: unpleasant_person, disagreeable_person
Meaning 3 NLTK ID: turkey.n.04
H

Meaning 0 NLTK ID: austria.n.01
Hypernyms: 
Meaning 0 NLTK ID: serbia.n.01
Hypernyms: 
Meaning 0 NLTK ID: ireland.n.01
Hypernyms: 
Meaning 1 NLTK ID: ireland.n.02
Hypernyms: 
Meaning 0 NLTK ID: lithuania.n.01
Hypernyms: 
Meaning 0 NLTK ID: latvia.n.01
Hypernyms: 
Meaning 0 NLTK ID: croatia.n.01
Hypernyms: 
Meaning 0 NLTK ID: slovakia.n.01
Hypernyms: 
Meaning 0 NLTK ID: estonia.n.01
Hypernyms: 
Meaning 0 NLTK ID: switzerland.n.01
Hypernyms: 
Meaning 0 NLTK ID: moldova.n.01
Hypernyms: 
Meaning 0 NLTK ID: belgium.n.01
Hypernyms: 
Meaning 0 NLTK ID: albania.n.01
Hypernyms: 
Meaning 0 NLTK ID: slovenia.n.01
Hypernyms: 
Meaning 0 NLTK ID: montenegro.n.01
Hypernyms: 
Meaning 0 NLTK ID: kosovo.n.01
Hypernyms: 
Meaning 0 NLTK ID: cyprus.n.01
Hypernyms: 
Meaning 1 NLTK ID: cyprus.n.02
Hypernyms: 
Meaning 0 NLTK ID: luxembourg-ville.n.01
Hypernyms: 
Meaning 1 NLTK ID: luxembourg.n.02
Hypernyms: 
Meaning 0 NLTK ID: andorra.n.01
Hypernyms: 
Meaning 0 NLTK ID: malta.n.01
Hypernyms: 
Meaning 1 NLTK I

...

Meaning 0 NLTK ID: cameroon.n.01
Hypernyms: 
Meaning 1 NLTK ID: cameroon.n.02
Hypernyms: 
Meaning 0 NLTK ID: canada.n.01
Hypernyms: 
Meaning 0 NLTK ID: chad.n.01
Hypernyms: paper
Meaning 1 NLTK ID: lake_chad.n.01
Hypernyms: 
Meaning 2 NLTK ID: chad.n.03
Hypernyms: 
Meaning 3 NLTK ID: chad.n.04
Hypernyms: Afroasiatic, Afro-Asiatic, Afroasiatic_language, Afrasian, Afrasian_language, Hamito-Semitic
Meaning 0 NLTK ID: china.n.01
Hypernyms: 
Meaning 1 NLTK ID: china.n.02
Hypernyms: porcelain
Meaning 2 NLTK ID: taiwan.n.01
Hypernyms: 
Meaning 3 NLTK ID: chinaware.n.01
Hypernyms: crockery, dishware
Meaning 0 NLTK ID: colombia.n.01
Hypernyms: 
Meaning 0 NLTK ID: comoros.n.01
Hypernyms: 
Meaning 0 NLTK ID: croatia.n.01
Hypernyms: 
Meaning 0 NLTK ID: denmark.n.01
Hypernyms: 
Meaning 0 NLTK ID: dominica.n.01
Hypernyms: 
Meaning 1 NLTK ID: dominica.n.02
Hypernyms: 
Meaning 0 NLTK ID: ecuador.n.01
Hypernyms: 
Meaning 0 NLTK ID: egypt.n.01
Hypernyms: 
Meaning 1 NLTK ID: egyptian_empire.n.01
Hypernym

Meaning 1 NLTK ID: area.n.02
Hypernyms: topic, subject, issue, matter
Meaning 2 NLTK ID: area.n.03
Hypernyms: body_part
Meaning 3 NLTK ID: sphere.n.01
Hypernyms: environment
Meaning 4 NLTK ID: area.n.05
Hypernyms: structure, construction
Meaning 5 NLTK ID: area.n.06
Hypernyms: extent
Meaning 0 NLTK ID: adelaide.n.01
Hypernyms: 
Meaning 0 NLTK ID: california.n.01
Hypernyms: 
Meaning 0 NLTK ID: city.n.01
Hypernyms: municipality
Meaning 1 NLTK ID: city.n.02
Hypernyms: administrative_district, administrative_division, territorial_division
Meaning 2 NLTK ID: city.n.03
Hypernyms: municipality
Meaning 0 NLTK ID: land.n.01
Hypernyms: real_property, real_estate, realty, immovable
Meaning 1 NLTK ID: land.n.02
Hypernyms: object, physical_object
Meaning 2 NLTK ID: domain.n.02
Hypernyms: region
Meaning 3 NLTK ID: land.n.04
Hypernyms: object, physical_object
Meaning 4 NLTK ID: country.n.02
Hypernyms: administrative_district, administrative_division, territorial_division
Meaning 5 NLTK ID: kingdom.n.

Meaning 2 NLTK ID: township.n.01
Hypernyms: administrative_district, administrative_division, territorial_division
Meaning 3 NLTK ID: town.n.04
Hypernyms: architect, designer
Meaning 0 NLTK ID: canada.n.01
Hypernyms: 
Meaning 0 NLTK ID: area.n.01
Hypernyms: region
Meaning 1 NLTK ID: area.n.02
Hypernyms: topic, subject, issue, matter
Meaning 2 NLTK ID: area.n.03
Hypernyms: body_part
Meaning 3 NLTK ID: sphere.n.01
Hypernyms: environment
Meaning 4 NLTK ID: area.n.05
Hypernyms: structure, construction
Meaning 5 NLTK ID: area.n.06
Hypernyms: extent
Meaning 0 NLTK ID: association.n.01
Hypernyms: organization, organisation
Meaning 1 NLTK ID: association.n.02
Hypernyms: social_activity
Meaning 2 NLTK ID: association.n.03
Hypernyms: union, unification
Meaning 3 NLTK ID: association.n.04
Hypernyms: memory, remembering
Meaning 4 NLTK ID: affiliation.n.01
Hypernyms: relationship
Meaning 5 NLTK ID: association.n.06
Hypernyms: relation
Meaning 6 NLTK ID: association.n.07
Hypernyms: chemical_process,

Meaning 3 NLTK ID: sphere.n.01
Hypernyms: environment
Meaning 4 NLTK ID: area.n.05
Hypernyms: structure, construction
Meaning 5 NLTK ID: area.n.06
Hypernyms: extent
Meaning 0 NLTK ID: member.n.01
Hypernyms: associate
Meaning 1 NLTK ID: member.n.02
Hypernyms: part, portion, component_part, component, constituent
Meaning 2 NLTK ID: extremity.n.01
Hypernyms: external_body_part
Meaning 3 NLTK ID: member.n.04
Hypernyms: unit, social_unit
Meaning 4 NLTK ID: penis.n.01
Hypernyms: erectile_organ
Meaning 0 NLTK ID: unite.v.01
Hypernyms: join, fall_in, get_together
Meaning 1 NLTK ID: unify.v.01
Hypernyms: integrate
Meaning 2 NLTK ID: unite.v.03
Hypernyms: have, feature
Meaning 3 NLTK ID: connect.v.03
Hypernyms: 
Meaning 4 NLTK ID: unite.v.05
Hypernyms: change, alter, modify
Meaning 5 NLTK ID: unite.v.06
Hypernyms: change, alter, modify
Meaning 6 NLTK ID: united.a.01
Hypernyms: 
Meaning 7 NLTK ID: joined.s.01
Hypernyms: 
Meaning 0 NLTK ID: congress.n.01
Hypernyms: legislature, legislative_assembl

Hypernyms: 
Meaning 0 NLTK ID: twelve.n.01
Hypernyms: large_integer
Meaning 1 NLTK ID: twelve.s.01
Hypernyms: 
Meaning 0 NLTK ID: twelve.n.01
Hypernyms: large_integer
Meaning 1 NLTK ID: twelve.s.01
Hypernyms: 
Meaning 0 NLTK ID: twelve.n.01
Hypernyms: large_integer
Meaning 1 NLTK ID: twelve.s.01
Hypernyms: 
Meaning 0 NLTK ID: twelve.n.01
Hypernyms: large_integer
Meaning 1 NLTK ID: twelve.s.01
Hypernyms: 
Meaning 0 NLTK ID: twelve.n.01
Hypernyms: large_integer
Meaning 1 NLTK ID: twelve.s.01
Hypernyms: 
Meaning 0 NLTK ID: twelve.n.01
Hypernyms: large_integer
Meaning 1 NLTK ID: twelve.s.01
Hypernyms: 
Meaning 0 NLTK ID: twelve.n.01
Hypernyms: large_integer
Meaning 1 NLTK ID: twelve.s.01
Hypernyms: 
Meaning 0 NLTK ID: city.n.01
Hypernyms: municipality
Meaning 1 NLTK ID: city.n.02
Hypernyms: administrative_district, administrative_division, territorial_division
Meaning 2 NLTK ID: city.n.03
Hypernyms: municipality
Meaning 0 NLTK ID: town.n.01
Hypernyms: municipality
Meaning 1 NLTK ID: town.n

In [46]:
hypernyms_list[1]

[['Algonquian', 'Algonquin'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['goose'],
 [],
 [],
 [],
 [],
 ['shore'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['retailer', 'retail_merchant'],
 ['maker', 'shaper'],
 [],
 [],
 [],
 [],
 ['carriage', 'equipage', 'rig'],
 [],
 [],
 ['infectious_disease'],
 [],
 [],
 [],
 ['municipality'],
 ['administrative_district',
  'administrative_division',
  'territorial_division'],
 ['municipality'],
 ['real_property', 'real_estate', 'realty', 'immovable'],
 ['object', 'physical_object'],
 ['region'],
 ['object', 'physical_object'],
 ['administrative_district',
  'administrative_division',
  'territorial_division'],
 ['sphere', 'domain', 'area', 'orbit', 'field', 'arena'],
 ['real_property', 'real_estate', 'realty', 'immovable'],
 ['people'],
 ['political_unit', 'political_entity'],
 [],
 ['occupation', 'business', 'job', 'line_of_work', 'line'],
 ['arrive', 'get', 'come'],
 ['arrive', 'get', 'come'],
 ['change', 'alter', 'modify'],
 ['bring',

In [47]:
len(hypernyms_list)

93

<h1> Final(To pass List) </h1>

<h1> Combining Name of All Attributes + Root Words + Superlative Phrase + Hypernyms </h1>

Combining all the above stated features and then dividing into training and testing dataset 

In [48]:
passing_list = []
for v in range(len(title_list_ref)): # for iterating through total number of records
    final_str = ''
    for h_values in hypernyms_list[v]: # for iterating through total number of hypernyms
        for hh in h_values:
            final_str = final_str+' '+hh
    final_str = final_str+ ' '+root_word_list[v]  #for combining root word list 
    for c_values in column_names[v]: # for iterating through column names
        final_str = final_str+' '+c_values
    final_str = final_str+ ' '+sup_phrases[v] # for combining superlative phrase
    passing_list.append(final_str)

In [49]:
passing_list[0]

' dynasty royalty royal_family royal_line royal_house bank municipality administrative_district administrative_division territorial_division municipality real_property real_estate realty immovable object physical_object region object physical_object administrative_district administrative_division territorial_division sphere domain area orbit field arena real_property real_estate realty immovable people political_unit political_entity occupation business job line_of_work line arrive get come arrive get come change alter modify bring convey take deliver drive_home arrive get come region topic subject issue matter body_part environment structure construction extent cities Rank City Land area Cum area Total area Population (2010) Population Density largest'

In [50]:
len(passing_list)

93

<h1> TFIDF and Word2Vec Embeddings for Training Dataset <h1>

In [51]:
t_counter = 0
training_str_list = []
testing_list = []
training_labels = []
for v in sup_phrases:
    if v:
        training_str_list.append(passing_list[t_counter])
        training_labels.append(v)
    else:
        testing_list.append(passing_list[t_counter])
    t_counter = t_counter+1

In [52]:
training_labels

['largest',
 'largest',
 'largest',
 'largest',
 'largest',
 'largest',
 'most',
 'most',
 'largest',
 'oldest',
 'largest',
 'largest',
 'largest',
 'oldest',
 'oldest',
 'largest',
 'largest',
 'most',
 'heaviest',
 'longest',
 'fastest',
 'fastest',
 'largest',
 'largest',
 'largest',
 'longest',
 'smallest',
 'largest',
 'longest',
 'longest',
 'wealthiest',
 'largest',
 'most',
 'longest',
 'greatest',
 'wealthiest',
 'largest',
 'highest',
 'highest',
 'most',
 'largest',
 'tallest',
 'heaviest',
 'youngest',
 'youngest',
 'youngest',
 'youngest',
 'oldest',
 'largest',
 'tallest',
 'youngest',
 'youngest']

In [53]:
training_str_list

[' dynasty royalty royal_family royal_line royal_house bank municipality administrative_district administrative_division territorial_division municipality real_property real_estate realty immovable object physical_object region object physical_object administrative_district administrative_division territorial_division sphere domain area orbit field arena real_property real_estate realty immovable people political_unit political_entity occupation business job line_of_work line arrive get come arrive get come change alter modify bring convey take deliver drive_home arrive get come region topic subject issue matter body_part environment structure construction extent cities Rank City Land area Cum area Total area Population (2010) Population Density largest',
 ' Algonquian Algonquin goose shore retailer retail_merchant maker shaper carriage equipage rig infectious_disease municipality administrative_district administrative_division territorial_division municipality real_property real_estat

In [54]:
testing_list

[' municipality administrative_district administrative_division territorial_division municipality rectangle regular_polygon number tract piece_of_land piece_of_ground parcel_of_land parcel shape form simpleton simple conservative conservativist artifact artefact hand_tool shape form multiply adjust conform adapt position match fit correspond check jibe gibe tally agree settle paddle row magnitude property filler situation state_of_affairs magnitude coat surface classify class sort assort sort_out separate change_magnitude squares City square City Country Area Dimensions; Notes Yearcompleted Photo ',
 ' porcelain crockery dishware edible_nut fish_genus paper Afroasiatic Afro-Asiatic Afroasiatic_language Afrasian Afrasian_language Hamito-Semitic canasta basket_rummy meld domestic_fowl fowl poultry unpleasant_person disagreeable_person poultry flop bust fizzle hot_pepper leather lacquerware lacquer lacquer Western_Malayo-Polynesian coin Italian domestic_fowl fowl poultry Asian Asiatic Tai

<h1> TFIDF </h1>

In [55]:
total_words=[]
for i in range(0,len(training_str_list)):
    tokens = nltk.word_tokenize(training_str_list[i])
    for w in tokens:
        total_words.append(w)
print(len(total_words))

3671


In [56]:
counts=Counter(total_words)

In [57]:
counts

Counter({'dynasty': 1,
         'royalty': 1,
         'royal_family': 1,
         'royal_line': 1,
         'royal_house': 1,
         'bank': 1,
         'municipality': 68,
         'administrative_district': 54,
         'administrative_division': 54,
         'territorial_division': 54,
         'real_property': 28,
         'real_estate': 28,
         'realty': 28,
         'immovable': 28,
         'object': 28,
         'physical_object': 28,
         'region': 47,
         'sphere': 14,
         'domain': 15,
         'area': 28,
         'orbit': 14,
         'field': 14,
         'arena': 14,
         'people': 20,
         'political_unit': 19,
         'political_entity': 19,
         'occupation': 14,
         'business': 15,
         'job': 14,
         'line_of_work': 14,
         'line': 15,
         'arrive': 42,
         'get': 42,
         'come': 42,
         'change': 18,
         'alter': 17,
         'modify': 17,
         'bring': 14,
         'convey': 14,
   

In [58]:
len(counts)

1332

In [59]:
vocab_obtained={j:i for i,j in enumerate(counts.keys())}

In [60]:
vectorize = TfidfVectorizer( vocabulary=vocab_obtained)
tfidf_matrix = vectorize.fit_transform(training_str_list)
tfidf_matrix

<52x1332 sparse matrix of type '<class 'numpy.float64'>'
	with 2463 stored elements in Compressed Sparse Row format>

<h1> Importing GloVE File </h1>

In [61]:
import io
words_l=[]
i=0
file_link = "/Users/datascience/Documents/itec_5205_project/glove.6B/glove.6B.50d.txt"
def load_glove_Model_processing(file_link):
    with io.open(file_link, encoding="utf8" ) as ff:
        content_l = ff.readlines()
    model_dict = {}
    for l in content_l:
        split_Line = l.split()
        word_l = split_Line[0]
        embedding_l = np.array([float(val) for val in split_Line[1:]])
        model_dict[word_l] = embedding_l
        words_l.append(word_l)
    print (len(model_dict)," words loaded!")
    return model_dict

model= load_glove_Model_processing(file_link) 

400000  words loaded!


In [62]:
scores={}
for i in range(0,len(training_str_list)):
    tokens= nltk.word_tokenize(training_str_list[i])
    for w in tokens:
        try :
            scores[w]=np.array(model[w])
        except : 
            scores[w]=np.zeros((1,50))

In [63]:
Word_Matrix=[np.reshape(scores[i],(1,50)) for i in scores.keys()]

In [64]:
word_matrix_embeddings=np.reshape(Word_Matrix,(len(Word_Matrix),50))

In [65]:
word_matrix_embeddings.shape

(1332, 50)

In [66]:
sparse_tfidf_matrix = sparse.csr_matrix(tfidf_matrix)

In [67]:
sparse_tfidf_matrix.shape

(52, 1332)

In [68]:
sparse_word_matrix = sparse.csr_matrix(word_matrix_embeddings)

In [69]:
sparse_word_matrix.shape

(1332, 50)

<h1> GloVe Training Matrix </h1>

In [70]:
final_matrix=np.dot(sparse_tfidf_matrix,sparse_word_matrix)

In [71]:
final_matrix.shape

(52, 50)

<h1> Preparing testing Data </h1>

<h2> TFIDF </h2>

In [72]:
test_total_words=[]
for i in range(0,len(testing_list)):
    tokens = nltk.word_tokenize(testing_list[i])
    for w in tokens:
        test_total_words.append(w)
print(len(test_total_words))

3455


In [73]:
test_counts=Counter(test_total_words)
len(test_counts)

920

In [74]:
test_vocab_obtained={j:i for i,j in enumerate(test_counts.keys())}

In [75]:
testvectorize = TfidfVectorizer( vocabulary=test_vocab_obtained)
test_tfidf_matrix = testvectorize.fit_transform(testing_list)
test_tfidf_matrix

<41x920 sparse matrix of type '<class 'numpy.float64'>'
	with 2065 stored elements in Compressed Sparse Row format>

<h2> Preparing Glove Matrix for Test Set </h2>

In [76]:
test_scores={}
for i in range(0,len(testing_list)):
    tokens= nltk.word_tokenize(testing_list[i])
    for w in tokens:
        try :
            test_scores[w]=np.array(model[w])
        except : 
            test_scores[w]=np.zeros((1,50))

In [77]:
test_Word_Matrix=[np.reshape(test_scores[i],(1,50)) for i in test_scores.keys()]
test_word_matrix_embeddings=np.reshape(test_Word_Matrix,(len(test_Word_Matrix),50))

In [78]:
test_word_matrix_embeddings.shape

(920, 50)

In [79]:
test_sparse_tfidf_matrix = sparse.csr_matrix(test_tfidf_matrix)
test_sparse_tfidf_matrix.shape

(41, 920)

In [80]:
test_sparse_word_matrix = sparse.csr_matrix(test_word_matrix_embeddings)

In [81]:
test_sparse_word_matrix.shape

(920, 50)

In [82]:
test_final_matrix=np.dot(test_sparse_tfidf_matrix,test_sparse_word_matrix)

In [83]:
test_final_matrix.shape

(41, 50)

<h1> Application of Models </h1>

In [84]:
from sklearn.ensemble import RandomForestClassifier

In [85]:
#Y_test = []

In [86]:
X_train = final_matrix
Y_train = training_labels
X_test = test_final_matrix

In [92]:
Y_test = ['largest','largest','largest'
'largest','largest','largest', 'largest', 'largest',
'largest', 'most', 'most', 'largest' ,'largest','biggest','most','most','largest'
'biggest','last','last','last','extreme','extreme'
,'extreme','largest', 'largest','largest','largest'
'largest','largest','largest', 'largest', 'largest','largest','largest','largest','largest'
'largest', 'largest','largest','equality','largest','largest','long','shortest']

In [87]:
r_f=RandomForestClassifier()
#start_time=time.time()
r_f.fit(X_train,Y_train)
value_pred_r_f = r_f.predict(X_test)
print(value_pred_r_f)

['largest' 'largest' 'largest' 'largest' 'largest' 'largest' 'largest'
 'largest' 'largest' 'largest' 'largest' 'largest' 'largest' 'largest'
 'largest' 'largest' 'largest' 'largest' 'largest' 'largest' 'largest'
 'most' 'largest' 'largest' 'largest' 'largest' 'largest' 'largest'
 'youngest' 'largest' 'largest' 'largest' 'largest' 'largest' 'largest'
 'largest' 'largest' 'largest' 'largest' 'youngest' 'tallest']


In [98]:
print("Accuracy after applying random forest",(accuracy_score(Y_test,value_pred_r_f)))

Accuracy after applying random forest 0.5365853658536586


<h1> Rough Trials </h1>

In [77]:
#from wikitables import import_tables
#tables = import_tables('List of largest California cities by land area')

In [39]:
#tables[0].name

'List of largest California cities by land area[0]'

In [44]:
#tables[0].head

['Rank',
 'City',
 'Land area',
 'Cum area',
 'Total area',
 'Population (2010)',
 'Population Density']

In [54]:
##for row in tables[0].rows:
  #  cc = ('{City}'.format(**row))

In [28]:
#import spacy
#from spacy import displacy
#from collections import Counter
#import en_core_web_sm
#nlp = en_core_web_sm.load()

In [123]:
#type(member_values[0])

In [46]:
#str1 = ' '.join(str(e) for e in member_values[0])

In [124]:
#str1

In [125]:
#pp = member_values[0][0]

In [87]:
#lg = ([(X.text, X.label_) for X in str1.ents])

In [89]:
#lg

[('Los Angeles', 'GPE'),
 ('California', 'GPE'),
 ('San Jose', 'GPE'),
 ('Lancaster Palm Springs Riverside', 'ORG'),
 ('Irvine Stockton', 'PERSON'),
 ('Oakland', 'GPE'),
 ('Thousand', 'GPE')]

In [134]:
#hypernyms_list = []

In [ ]:
#from nltk.corpus import wordnet as wn
#from itertools import chain
 
#input_word = input("Enter word to get hyponyms and hypernyms: ")

#for i,j in enumerate(wn.synsets('cities')):
#    print ("Meaning",i, "NLTK ID:", j.name())
#    print ("Hypernyms:", ", ".join(list(chain(*[l.lemma_names() for l in j.hypernyms()]))))
#    ll = (list(chain(*[l.lemma_names() for l in j.hypernyms()])))

In [49]:
#import wikipedia as wp

#html_file = wp.page("List of largest California cities by land area").html().encode("UTF-8")
#df = pd.read_html(html_file)[0]
#df.to_csv('table1.csv',header=0,index=False)
#print (df)

In [24]:
##import requests
##website_url = requests.get('https://en.wikipedia.org/wiki/List_of_Asian_countries_by_area').text

In [45]:
##from bs4 import BeautifulSoup
##soup = BeautifulSoup(website_url,'lxml')
##print(soup.prettify())

In [46]:
##My_table = soup.find('table',{'class':'wikitable sortable'})
##My_table

In [47]:
#import wikipedia as wp
 
#Get the html source
#html = wp.page("List_of_highest-grossing_films").html().encode("UTF-8")
#df = pd.read_html(html)[0]
#df.to_csv('india_cities_popl.csv',header=0,index=False)
#print (df)

In [48]:
##wikitablescrape --url="https://en.wikipedia.org/wiki/List_of_largest_California_cities_by_land_area" --header="films by year" | tee >(head -1) >(tail -5) >/dev/null

<h1> Loading Templates </h1>

In [3]:
templates = pd.read_csv('template.csv')

In [4]:
templates.head()

,{entity} is the {rank} largest City
0,{entity} is the {rank} largest City
1,{entity} is the {rank} largest Municipality
2,{entity} is the {rank} largest City Square
3,{entity} is the {rank} largest Sovereign State...
4,{entity} is the {rank} largest Country or Teri...
